In [ ]:
'''

掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

In [1]:
'''
資料 mapping 至google drive
把資料寫在/content/drive
即可保存在 google drive內
>> 為什麼要保存在drive?
因為Colab會把資料清空，在開發階段，會建議掛載google drive, 讓資料能夠被保存下來
!! 只有在開發階段使用，正式環境存東西，不應存在這
'''
# 掛載google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
流程解析
1. 前置作業：引用套件
2. 建立app主邏輯(01.http.ipynb)
3. 生成傳給用戶的消息
4. 設置：後續收到的消息，要怎麼處理
5. 運行app
'''


In [7]:
# 安裝套件
# !pip install line-bot-sdk flask flask-ngrok
# !pip install pyngrok

!curl -L https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -o ngrok.zip && unzip ngrok.zip && rm ngrok.zip
!./ngrok authtoken 2Miom0KAoRAXk9dAQyUBIBXEQg7_7Bp4cz6ti7QVh5ayzZJNQ
#----
# 測試
# ngrok成功，顯示：Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.2M  100 13.2M    0     0  25.8M      0 --:--:-- --:--:-- --:--:-- 25.8M
Archive:  ngrok.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [9]:
'''
建置主程序app
建置handler與 line_bot_api
# Step1. 準備好app
# Step2. 準備handler, 用來接收Line傳來的一切事物，順便做驗證，就像小秘書過濾信件
# Step3. 準備line_bot_api, 有些業務要跟Line溝通，都會透過line_bot_api回信、取個資
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
#
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("svZ59+2NW9n9kRK6F5Dprw7lq5zr/x0gYftKxq7eWD3yXFD+qob9Dr3vPKbVj7UK2K2gx/Ne2bEhedLvCnuOw8mfOFK7jTU+t0TD3pRYjLX0q4Mx+sLtOjCH1iER6AbAdS3P4KncJl3tpZcx8RGIggdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("d8786afd5abc6a8efb667ab3dae71146")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value (LINE 加密簽章)
    signature = request.headers['X-Line-Signature']

    # get request body as text (用戶傳來的內容)
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log # 存在google drive上，取名為ai-event.log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    # 把消息交給handler(小秘書)做驗證，會依照不同Event, 做不同處理
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:

'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料(用line_bot_api跟user溝通，並調用get_profile取得個資)
    # event = 每次傳來的消息; source = 來源; user_id = 發消息的用戶id
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內, 存在google drive內，users.txt的檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # LINE bot回覆文字消息與圖片消息
    # line_bot_api >> LINE平台 >> user
    # [TextSendMessage(<回應文字>)]
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('安安，你的個資已被我紀錄了')]
    )

In [ ]:
# 運行主程序
app.run()